In [135]:
import numpy as np
import plotly.express as plx
import pandas as pd 

In [136]:
def sigmoid(x:any)->float:
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x:any)->float:
    return x * (1 - x)


class NeuralNetwork:
    def __init__(self, input_size:int, hidden_size:int, output_size:int, learning_rate:float = 0.1):
        self.input_size  = input_size
        self.hidden_size  = hidden_size
        self.output_size  = output_size
        self.learning_rate = learning_rate
        np.random.seed(110)
        self.hidden_weights = np.random.uniform(size=(input_size, hidden_size))
        self.hidden_bias = np.zeros((1, hidden_size))
        self.output_weights = np.random.uniform(size=(hidden_size, output_size))
        self.output_bias = np.zeros((1, output_size))

    def forward_pass(self,inputs:np.ndarray)\
            ->tuple[np.ndarray,np.ndarray]:
        hidden_layer_activation = np.dot(inputs, self.hidden_weights) + self.hidden_bias    
        hidden_layer_output = sigmoid(hidden_layer_activation)
    
        output_layer_activation = np.dot(hidden_layer_output, self.output_weights) + self.output_bias
        predicted_output = sigmoid(output_layer_activation)
        return predicted_output, hidden_layer_output

    def backward_pass(self,expected_output:np.ndarray, predicted_output:np.ndarray,hidden_layer_output:np.ndarray)\
            -> tuple[np.ndarray, np.ndarray, np.float64]:
        error = expected_output - predicted_output
        d_predicted_output = error * sigmoid_derivative(predicted_output)
        error_hidden_layer = d_predicted_output.dot(self.output_weights.T)
        d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output) 
        sse = sum(error ** 2)
        return d_predicted_output, d_hidden_layer,sse
    
    
    def train(self,X:np.ndarray,y:np.ndarray,ep:int=1000)\
            ->tuple[np.ndarray,list]:
        predicted_output = np.NAN
        sse = []
        for epoch in range(ep):
            predicted_output, hidden_layer_output = self.forward_pass(X)
            d_predicted_output, d_hidden_layer,error = self.backward_pass(y, predicted_output, hidden_layer_output)
            sse.append(np.max(error)) 
            
            self.output_weights += hidden_layer_output.T.dot(d_predicted_output) * self.learning_rate
            self.output_bias += np.sum(d_predicted_output, axis=0, keepdims=True) * self.learning_rate
            self.hidden_weights += X.T.dot(d_hidden_layer) * self.learning_rate
            self.hidden_bias += np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate
        return predicted_output,sse
    
    def predict(self, x_data:np.ndarray)\
            ->np.ndarray:
        predicted_output, hidden_layer_output = self.forward_pass(x_data)
        return np.array(list(map(np.round,predicted_output)))


In [137]:
inp_nod = 7
hid_nod = 4
out_nod = 10
X_train = np.array([
    np.array([1,1,1,0,1,1,1]), # 0
    np.array([0,0,1,0,0,1,0]), # 1
    np.array([1,0,1,1,1,0,1]), # 2
    np.array([1,0,1,1,0,1,1]), # 3
    np.array([0,1,1,1,0,1,0]), # 4
    np.array([1,1,0,1,0,1,1]), # 5
    np.array([1,1,0,1,1,1,1]), # 6
    np.array([1,0,1,0,0,1,0]), # 7 
    np.array([1,1,1,1,1,1,1]), # 8
    np.array([1,1,1,1,0,1,1]), # 9
])
Y_train = np.zeros((10,10))
for i in range(10):
    Y_train[i] = [0]*i + [1] + [0] *(9-i)


In [151]:
epochs = 1000
model = NeuralNetwork(input_size=inp_nod,hidden_size=hid_nod,output_size=out_nod)
probka,SQE = model.train(X=X_train,y=Y_train,ep=epochs)

dp = pd.DataFrame({"x":list(range(len(SQE))),"y":map(np.float64,SQE)})

fig = plx.line(data_frame=dp,
               x="x",y="y",
               title="Изменение суммы квадратов ошибок с изменением эпохи при значении learning_rate = 0.1",
               labels={"x":"Эпоха","y":"SSE"})

fig.show()

In [147]:
epochs = 1000
SQE_lr = []
for lr in np.linspace(0,1,1000):
    model = NeuralNetwork(input_size=inp_nod,hidden_size=hid_nod,output_size=out_nod,learning_rate=lr)
    probka,SQE = model.train(X=X_train,y=Y_train,ep=epochs)
    SQE_lr.append(np.mean(SQE))
SQE_lr =list(map(np.float64,SQE_lr))
fig = plx.line(x=list(map(lambda x:x/1000,range(len(SQE_lr)))),
                      y=SQE_lr,
               title=f"Изменение суммы квадратов ошибок с изменением learning_rate при значении эпох {epochs}",
               labels={"x":"learning_rate","y":"SSE"})
fig.show()

In [148]:
min_SSE = min(SQE_lr)
print(min_SSE)

0.2839187184813591


In [156]:
fp = SQE_lr[0]
lr = 0.1
for i in range(1,len(SQE_lr)):
    delt = fp - SQE_lr[i]
    fp = SQE_lr[i]
    if delt < 0.001:
        lr = i/1000
        print(f"Самый оптимальный learning_rate это :{lr} \n При нем ошибка SSE:{SQE_lr[i]}")
        break


Самый оптимальный learning_rate это :0.074 
 При нем ошибка SSE:0.9835557477936964


In [157]:
lr = 0.95

In [158]:
epochs = 1000
model = NeuralNetwork(input_size=inp_nod,hidden_size=hid_nod,output_size=out_nod,learning_rate=lr)
probka,SQE = model.train(X=X_train,y=Y_train,ep=epochs)
dp = pd.DataFrame({"x":list(range(len(SQE))),"y":map(np.float64,SQE)})
fig = plx.line(data_frame=dp,
               x="x",y="y",
               title=f"Изменение суммы квадратов ошибок с изменением эпохи \nпри значении learning_rate = {lr}",
               labels={"x":"Эпоха","y":"SSE"})

fig.show()

In [159]:
pd.DataFrame(zip(model.predict(X_train[::3]),Y_train[::3]))

,0,1
0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
